In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import chess.pgn
import numpy as np
from IPython.display import display, clear_output
import torch
from fastai2.data import *
from fastai2.basics import *
from fastai2.callback.all import *
from models import Model1

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
def get_bitboard(board):
    '''
    params
    ------
    board : chess.pgn board object
        board to get state from
    returns
    -------
    bitboard representation of the state of the game
    64 * 6 + 5 dim binary numpy vector
    64 squares, 6 pieces, '1' indicates the piece is at a square
    5 extra dimensions for castling rights queenside/kingside and whose turn
    '''
    bitboard = np.zeros(64*6*2+5)
    piece_idx = {'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5}
    for i in range(64):
        if board.piece_at(i):
            color = int(board.piece_at(i).color) + 1
            bitboard[(piece_idx[board.piece_at(i).symbol().lower()] + i * 6) * color] = 1
    bitboard[-1] = int(board.turn)
    bitboard[-2] = int(board.has_kingside_castling_rights(True))
    bitboard[-3] = int(board.has_kingside_castling_rights(False))
    bitboard[-4] = int(board.has_queenside_castling_rights(True))
    bitboard[-5] = int(board.has_queenside_castling_rights(False))
    return bitboard

In [5]:
def get_result(game):
    result = game.headers['Result']
    result = result.split('-')
    if result[0] == '1':
        return 1
    elif result[0] == '0':
        return -1
    else:
        return 0

In [6]:
def pgn_to_np(path):
    games = open(path)
    bitboards = []
    labels = []
    for i in progress_bar(range(100)):
        game = chess.pgn.read_game(games)
        try:
            result = get_result(game)
            board = game.board()
            for move in game.mainline_moves():
                board.push(move)
                bitboard = get_bitboard(board)

                bitboards.append(bitboard)
                labels.append(result)
        except:
            print(f"{i} games in file")
            break
    return torch.FloatTensor(bitboards), torch.FloatTensor(labels)

In [7]:
class ChessPosition(Transform):
    def __init__(self, bitboards, labels):
        self.bitboards, self.labels = bitboards, labels
        
    def encodes(self, i):
        return (bitboards[i], labels[i].unsqueeze(-1))

In [8]:
bitboards, labels = pgn_to_np('../data/games.pgn'); 
print(bitboards.shape)
bitboards

torch.Size([6679, 773])


tensor([[0., 0., 0.,  ..., 1., 1., 0.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 0.],
        [0., 0., 0.,  ..., 1., 1., 1.]])

In [9]:
cp = ChessPosition(bitboards, labels)
pipe = Pipeline([cp])
splits = RandomSplitter()(range_of(pipe(0)[0]))
tls = TfmdLists(range_of(pipe(0)[0]), pipe, splits=splits)
dls = tls.dataloaders(bs=8, device=device.type, num_workers=0)

In [12]:
dls.one_batch()

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 1., 1., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[-1.],
         [ 1.],
         [-1.],
         [ 1.],
         [ 1.],
         [ 0.],
         [ 1.],
         [ 1.]]))

In [24]:
learn = Learner(
    dls, 
    Model1(), 
    loss_func=nn.BCEWithLogitsLoss(), 
    opt_func=partial(Adam), 
)
learn.path = Path('..')

In [25]:
learn.fit(3)

epoch,train_loss,valid_loss,time
0,0.193317,0.144507,00:01
1,0.133239,0.110641,00:01
2,0.080116,0.103511,00:01


In [26]:
learn.export(fname='export.pkl')

AttributeError: new_empty

In [15]:
learn.model(tls[0][0].unsqueeze(0))

tensor([[-0.9054]], grad_fn=<TanhBackward>)

In [16]:
learn.predict(tls[0][0].unsqueeze(0))

TypeError: 'NoneType' object is not iterable

In [104]:
learn.save('model')